# Ejercicio 1

Para esta seccion, seleccionamos un corpus de la biblioteca nltk. Para este ejercicio seleccionamos el corpus **cess_es**. Este corpus consiste un texto de noticias en espanol

In [31]:
import re
import os
import pandas as pd
import numpy as np
from collections import Counter

from preprocesador import *
from subword import *
from frecuencias import *

os.environ["NLTK_DATA"]="datos_nltk/"
import  nltk
from nltk.corpus import stopwords

## Paso 1
Nos aseguramos de tener disponible el corpus **cess_esp**. Aprovechamos la funcionalidad de nltk para obtener las palabras del corpus. Estas palabras funcionaran como nuestros tokens.
Notemos que el corpus tiene un total de __192686__ palabra (tokens)



In [2]:
## Seleccionamos un corpus en español que se encuentra en nltk
from nltk.corpus import cess_esp

nltk.download("cess_esp")
corpus_tokens = cess_esp.words()

print("Numero de tokens en el corpus:", len(corpus_tokens))

[nltk_data] Downloading package cess_esp to /home/joel/nltk_data...
[nltk_data]   Package cess_esp is already up-to-date!


Numero de tokens en el corpus: 192686


## Paso 2
Aplicamos un pre procesamiento a todos los tokens de nuestro corpus. Los pasos a seguir en nuestro pre procesamiento son los siguientes:
- Pasamos todos los tokens a minusculas
- Eliminamos cualquier signo de puntacion en los tokens, es decir, eliminamos cualquier signo que no sea una letra del alfabeto en español o número
- Eliminamos todos los tokens que sean identificados como _stopword_. Para la lista de _stopword_ utilizamos la lista encontrada en ntlk para el lenguaje de español
- Finalmente eliminamos acentos. En este paso, para cada token, sustituimos sus vocales acentuadas por la misma vocal sin acento.



In [85]:
# Dado nuestro corpus tokenizado, normalizamos el texto:
# Removiendo acentos, puntuacion y stopwords

nltk.download('stopwords')
stopwords_list = stopwords.words('spanish')

#print(stopwords_list)
tokens_normalizados = pre_procesar(corpus_tokens, stopwords_list)

print("Numero de tokens, despues del preprocesamiento:", len(tokens_normalizados))

[nltk_data] Downloading package stopwords to /home/joel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Numero de tokens, despues del preprocesamiento: 86642


## Paso 3

Dado nuestros tokens normalizados, procedemos a realizar a obtener sub words utilizando BPE.

In [86]:
# Obtenemos las frecuencias de los tipos de cada subword utilizando bpe y tambien las reglas
freqs_bpe, tokens_bpe = aplicar_byte_pair_encoding(tokens_normalizados, 500)
tokens_bpe = [palabra.split() for palabra in tokens_bpe]
tokens_bpe = [x for xs in tokens_bpe for x in xs]


In [5]:
# Tenemos el texto tokenizado
print(tokens_bpe)

['grupo', 'estatal', 'elec', 'tri', 'ci', 'te', 'de', 'fran', 'ce', 'e', 'd', 'f', 'anuncio', 'hoy', 'jue', 'ves', 'co', 'mp', 'ra', '5', '1', 'porciento', 'empresa', 'mexi', 'cana', 'elec', 'tri', 'ci', 'dadagui', 'lade', 'al', 'ta', 'mira', 'e', 'a', 'a', 'cre', 'ada', 'japones', 'mi', 't', 'subis', 'hi', 'cor', 'poration', 'ponerenmar', 'cha', 'central', 'gas', '4', '9', '5', 'me', 'ga', 'vatios', 'por', 'ta', 'vo', 'z', 'e', 'd', 'f', 'expli', 'co', 'efe', 'proye', 'cto', 'cons', 'truc', 'cion', 'al', 'ta', 'mira', '2', 'nor', 'te', 'ta', 'mpi', 'co', 'pre', 've', 'uti', 'lizacion', 'gas', 'natural', 'co', 'mbus', 'ti', 'ble', 'princi', 'pal', 'central', 'cic', 'lo', 'co', 'mbinado', 'debe', 'e', 'mpe', 'zar', 'fun', 'cionar', 'mayo', 'del', '200', '2', 'elec', 'tri', 'ci', 'dad', 'produci', 'da', 'pasara', 're', 'd', 'elec', 'tri', 'ca', 'publi', 'ca', 'mexi', 'co', 'en', 'vir', 'tu', 'd', 'de', 'acuerdo', 'venta', 'ener', 'gia', 'e', 'a', 'a', 'comision', 'fe', 'deral', 'de', 'el

In [6]:
# Tabla para ver las frecuencias de las palabras
tabla_frecuencias_bpe = crear_tabla_frecuencias(freqs_bpe)
print(tabla_frecuencias_bpe)


        palabra  frecuencia
0             a        2253
1             o        2096
2            de        2036
3            co        1816
4            to        1574
...         ...         ...
9867    eratica           1
9868       raya           1
9869  chabacana           1
9870       arau           1
9871    coronar           1

[9872 rows x 2 columns]


In [35]:
#Convertimos la tabla en un diccionario para usarlo en el modelo
palabras = tabla_frecuencias_bpe['palabra']
frecuencias = tabla_frecuencias_bpe['frecuencia']
dic_palabras = {}  
for i in range(len(frecuencias)):
    dic_palabras[palabras[i]] = frecuencias[i]

In [36]:
ETIQUETA_UNK = "<UNK>"
def indices_palabras(palabras_frecuencias: dict):
    """Calcula los indices de las palabras dadas sus frecuencias

    Parameters
    ----------
    palabras_frecuencias : dict
        Diccionario donde las llaves son las palabras y los valores sus frecuencias

    Returns
    -------
    dict,dict
        Diccionarios uno a uno de palabras a indices y viceversa
    """
    result = {}
    for ind, palabra in enumerate(palabras_frecuencias.keys()):
        # Ocurrio un Happax legomena o singleton
        if palabras_frecuencias[palabra] == 1:
            # Indice temporal para UNK
            result[ETIQUETA_UNK] = len(palabras_frecuencias)
        else:
            result[palabra] = ind

    return {word: idx for idx, word in enumerate(result.keys())}, {idx: word for idx, word in enumerate(result.keys())}

In [38]:
#Se obtienen los índices de cada palabra
palabras_a_indices, indices_a_palabras = indices_palabras(dic_palabras)
print(palabras_a_indices)
print(indices_a_palabras)

{'a': 0, 'o': 1, 'de': 2, 'co': 3, 'to': 4, 'do': 5, 're': 6, 'ta': 7, 'e': 8, 'te': 9, 'con': 10, 'i': 11, 'ca': 12, 'se': 13, 'di': 14, 'da': 15, 'mo': 16, 'al': 17, 's': 18, 'dad': 19, 'c': 20, 'mi': 21, 'cion': 22, 'ma': 23, 't': 24, 'en': 25, 'l': 26, 'ar': 27, 'es': 28, 'p': 29, 'le': 30, 'in': 31, 'li': 32, 'si': 33, 'z': 34, 'd': 35, 'mente': 36, 'ex': 37, 'f': 38, 'ti': 39, 'fi': 40, 'go': 41, 'cia': 42, 'g': 43, 'ri': 44, 'cu': 45, 'k': 46, 'su': 47, 'vi': 48, 'lo': 49, 'dos': 50, 'que': 51, 'car': 52, 'per': 53, 'pro': 54, 'me': 55, 'des': 56, 've': 57, 'ci': 58, 'tas': 59, 'ce': 60, 'an': 61, 'mos': 62, '2': 63, 'ter': 64, 'pe': 65, 'os': 66, 'tos': 67, 'pi': 68, 'mas': 69, 'mar': 70, 'm': 71, 'tar': 72, 'ro': 73, 'por': 74, 'ra': 75, 'u': 76, '7': 77, 'ver': 78, 'so': 79, 'za': 80, 'hoy': 81, 'man': 82, '4': 83, 'lle': 84, 'no': 85, '3': 86, '6': 87, 'y': 88, 'la': 89, 'tan': 90, 'jo': 91, 'miento': 92, 'cos': 93, 'tu': 94, '5': 95, 'vo': 96, 'ga': 97, 'dis': 98, '8': 99, 

In [39]:
def get_id(palabra_a_id: dict, palabra: str) -> int:
    """Obtiene el id de una palabra dada

    Si no se encuentra la palabra se regresa el id
    del token UNK

    Parameters
    ----------
    palabra_a_id : dict
        Diccionario de palabras a indices
    palabra : str
        Palabra a buscar

    Returns
    -------
    int
        Indice de la palabra
    """
    id_unknow = palabra_a_id[ETIQUETA_UNK]
    return palabra_a_id.get(palabra, id_unknow)

## Paso 4
Definimos los bigramas a usar para entrenar nuestra red neuronal

In [83]:
def lista_bigramas(corpus:list, palabras_a_indices:dict) -> list:
    """Obtiene bigramas para usar en un modelo de red neuronal
    Parameters
    ----------
    corpus : list
        corpus a procesar
    indices_a_palabras : dict
        Diccionario de palabras a indices
    Returns
    -------
    list
        Conjunto de bigramas
    """
    bigramas = []
    for i in range(len(corpus)-1):
        bigrama = [get_id(palabras_a_indices, corpus[i]), get_id(palabras_a_indices, corpus[i+1])]
        bigramas.append(bigrama)
    return bigramas

In [84]:
#Obtenemos los bigramas
bigramas = lista_bigramas(tokens_normalizados, palabras_a_indices)
print(bigramas)

[[372, 2415], [2415, 6038], [6038, 6038], [6038, 708], [708, 81], [81, 6038], [6038, 6038], [6038, 6038], [6038, 607], [607, 6038], [6038, 6038], [6038, 6038], [6038, 6038], [6038, 1539], [1539, 6038], [6038, 6038], [6038, 1159], [1159, 266], [266, 6038], [6038, 6038], [6038, 6038], [6038, 6038], [6038, 6038], [6038, 732], [732, 6038], [6038, 6038], [6038, 6038], [6038, 6038], [6038, 6038], [6038, 6038], [6038, 6038], [6038, 266], [266, 1515], [1515, 6038], [6038, 6038], [6038, 1159], [1159, 6038], [6038, 6038], [6038, 481], [481, 6038], [6038, 6038], [6038, 6038], [6038, 6038], [6038, 6038], [6038, 2516], [2516, 6038], [6038, 6038], [6038, 6038], [6038, 6038], [6038, 6038], [6038, 449], [449, 507], [507, 6038], [6038, 6038], [6038, 6038], [6038, 6038], [6038, 2517], [2517, 670], [670, 102], [102, 6038], [6038, 1777], [1777, 6038], [6038, 6038], [6038, 1929], [1929, 6038], [6038, 6038], [6038, 6038], [6038, 6038], [6038, 6038], [6038, 6038], [6038, 6038], [6038, 6038], [6038, 6038], [6